In [17]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
#getting a list of S&P500 stocks
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df.to_csv('S&P500-Info.csv')

In [3]:
#generating a list of strings in dataframe column for symbol names
stocks=df['Symbol'].apply(lambda x: str(x))

In [5]:
#appending tickers to sp500 list
sp500 = []
for item in stocks:
    sp500.append(item)

In [6]:
IPGP = yf.Ticker("IPGP")

In [7]:
dataframe = pd.Series(IPGP.info, name = "IPGP").to_frame().T

In [8]:
#pull data from yFinance and append to dataframe
%%time

for item in sp500:
    try:
        dataframe.loc["{}".format(item)] = pd.Series(yf.Ticker(item).info)  
    except:
        print("Error for the following symbol : {} !".format(item))

Error for the following symbol : BRK.B !
Error for the following symbol : BF.B !
Wall time: 31min 10s


In [9]:
list(dataframe.columns)


['zip',
 'sector',
 'fullTimeEmployees',
 'longBusinessSummary',
 'city',
 'phone',
 'state',
 'country',
 'companyOfficers',
 'website',
 'maxAge',
 'address1',
 'fax',
 'industry',
 'previousClose',
 'regularMarketOpen',
 'twoHundredDayAverage',
 'trailingAnnualDividendYield',
 'payoutRatio',
 'volume24Hr',
 'regularMarketDayHigh',
 'navPrice',
 'averageDailyVolume10Day',
 'totalAssets',
 'regularMarketPreviousClose',
 'fiftyDayAverage',
 'trailingAnnualDividendRate',
 'open',
 'averageVolume10days',
 'expireDate',
 'yield',
 'algorithm',
 'dividendRate',
 'exDividendDate',
 'beta',
 'circulatingSupply',
 'startDate',
 'regularMarketDayLow',
 'priceHint',
 'currency',
 'trailingPE',
 'regularMarketVolume',
 'lastMarket',
 'maxSupply',
 'openInterest',
 'marketCap',
 'volumeAllCurrencies',
 'strikePrice',
 'averageVolume',
 'priceToSalesTrailing12Months',
 'dayLow',
 'ask',
 'ytdReturn',
 'askSize',
 'volume',
 'fiftyTwoWeekHigh',
 'forwardPE',
 'fromCurrency',
 'fiveYearAvgDividendYi

In [131]:
#creating a df with the following fundamentals
new = dataframe[[
 'beta',
 'trailingPE',
 'trailingEps',                
 'marketCap',
 'priceToSalesTrailing12Months',
 'profitMargins',
 'enterpriseToEbitda',
 'enterpriseToRevenue',
 'forwardEps',
 'bookValue',
 'trailingEps',
 'priceToBook',
 'enterpriseValue',
 'earningsQuarterlyGrowth',
 'sharesOutstanding'
]]

In [113]:
new.to_csv('new')

In [132]:
#Create Dataset and Target Variable
y=new['marketCap']
X=new.drop(['marketCap'], axis=1).fillna(0)

In [112]:
X.to_csv('X')

In [133]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_trans = sc.fit_transform(X)

In [134]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_trans, y, test_size=0.2, random_state=42)

In [135]:
len(X_train), len(y_train)

(402, 402)

In [136]:
from sklearn.ensemble import GradientBoostingRegressor as GBR

In [137]:
%%time
estimator = GBR(loss='huber', n_estimators=100, random_state=42)
estimator.fit(X_train, y_train)

Wall time: 248 ms


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='huber',
                          max_depth=3, max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [138]:
estimator.score(X_test, y_test)

0.9585307573061687